## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [5]:
data = range(1,1001)
len(data)
#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [6]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [7]:
rdd = sc.parallelize(data,numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [8]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [10]:
# count
rdd.count()

1000

In [12]:
# first
rdd.first()

1

In [14]:
# take
rdd.take(20)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [15]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [16]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [17]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([5.75719961e-02, 1.98137274e+00, 2.24257752e+00, 1.81396998e+00,
       2.18835580e+00, 5.37281539e+00, 6.07902945e+00, 5.04756587e+00,
       2.85877826e+00, 6.64590687e+00, 1.29755878e+00, 1.02098273e+01,
       8.42485989e+00, 6.94298199e+00, 2.33940278e+00, 1.28514221e+00,
       5.32904296e+00, 7.64905329e+00, 1.20400774e+01, 2.52623611e-01,
       2.06585876e+01, 5.59352344e+00, 1.03856545e+01, 8.98833660e+00,
       5.91578415e+00, 1.12892537e+00, 6.52742490e-01, 3.15699001e+00,
       1.54716480e+01, 1.99824400e+01, 2.33215590e+00, 3.35396659e+00,
       2.24738359e+01, 7.66654105e+00, 2.07854848e+01, 1.98391843e+01,
       1.29731287e+01, 3.74001498e+01, 2.24013236e+01, 1.98498332e+01,
       2.17848862e+00, 2.41243918e+00, 3.63261674e+01, 2.72324929e+01,
       2.68588070e+01, 2.61572806e+01, 2.79296777e+01, 4.31631578e+01,
       4.42429797e+01, 4.08563152e+01, 2.74272211e+00, 3.32447130e+01,
       1.06540333e+01, 1.97451394e+01, 5.49977267e+01, 2.22365250e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [21]:
price_items = sc.parallelize(sales_figures,numSlices=10)
price_items.top(5)

[962.9352395200935,
 959.5367697397678,
 909.9165589818957,
 891.7809991304232,
 888.1262881471541]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [22]:
def sales_tax(num):
    return 0.92*num

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [24]:
# perform action to retrieve rdd values
revenue_minus_tax.top(5)

[885.9004203584861,
 882.7738281605864,
 837.1232342633441,
 820.4385191999894,
 817.0761850953818]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [39]:
discounted = revenue_minus_tax.map(lambda x: 0.9*x)

In [40]:
discounted.top(5)

[797.3103783226376,
 794.4964453445277,
 753.4109108370097,
 738.3946672799905,
 735.3685665858436]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [41]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(5)

[797.3103783226376,
 794.4964453445277,
 753.4109108370097,
 738.3946672799905,
 735.3685665858436]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [42]:
price_items.toDebugString()

b'(10) ParallelCollectionRDD[12] at parallelize at PythonRDD.scala:195 []'

In [43]:
discounted.toDebugString()

b'(10) PythonRDD[31] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[12] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [37]:
mapped = price_items.map(lambda x: (x, 0.9*0.92*x))
print(mapped.count())
print(mapped.take(10))

1000
[(0.05757199613708597, 0.04766961280150719), (1.981372738270675, 1.640576627288119), (2.2425775208625143, 1.856854187274162), (1.81396997931909, 1.5019671428762067), (2.188355799831427, 1.8119586022604217), (5.372815390231736, 4.448691143111878), (6.079029446669579, 5.033436381842412), (5.047565868699196, 4.1793845392829345), (2.8587782618727484, 2.367068400830636), (6.645906868565479, 5.502810887172218)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [38]:
flat_mapped = price_items.flatMap(lambda x: (x, 0.9*0.92*x))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.05757199613708597, 0.04766961280150719, 1.981372738270675, 1.640576627288119, 2.2425775208625143, 1.856854187274162, 1.81396997931909, 1.5019671428762067, 2.188355799831427, 1.8119586022604217]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [44]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)

# calculate total remaining in inventory 
selected_items.count()

286

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [45]:
selected_items.reduce(lambda x,y :x + y)

132581.09681654006

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [46]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(22, 308.73942572075794),
 (42, 308.8860297592994),
 (43, 306.08517382366995),
 (48, 322.8651698779613),
 (2, 315.81167025528543),
 (8, 303.98696823869636),
 (48, 307.2829273266609)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [51]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y: x+y)
total_spent.take(7)

[(50, 2154.828212613725),
 (10, 1230.5923016266145),
 (20, 2503.9981054485906),
 (30, 1945.2656818896248),
 (40, 1944.1052143159877),
 (1, 4747.818616427754),
 (31, 2308.267537163193)]

In [53]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(12, 8015.412930954466),
 (19, 4830.606529704424),
 (1, 4747.818616427754),
 (32, 3994.2387832623785),
 (3, 3975.3427147313505),
 (41, 3909.9180766381373),
 (26, 3664.704049355362),
 (16, 3654.6210318355015),
 (35, 3579.0858198976043),
 (24, 3559.038605548701),
 (49, 3521.937523480567),
 (44, 3466.2338004402186),
 (22, 3306.504582395345),
 (28, 3254.5690365213577),
 (5, 3019.727155617132),
 (29, 3013.8470137367913),
 (7, 3007.1520792351394),
 (47, 2982.605583857763),
 (14, 2967.447685510173),
 (42, 2960.717881627648),
 (21, 2781.8267313165434),
 (4, 2686.043827146236),
 (36, 2603.6323579461828),
 (46, 2551.395163748427),
 (20, 2503.9981054485906),
 (8, 2432.570009985912),
 (11, 2351.714960370712),
 (31, 2308.267537163193),
 (50, 2154.828212613725),
 (33, 2153.8348170219588),
 (38, 2146.2474672131025),
 (13, 2097.569700404136),
 (6, 2064.941849722117),
 (30, 1945.2656818896248),
 (40, 1944.1052143159877),
 (15, 1930.8018130466894),
 (45, 1901.2711936328028),
 (23, 1896.1618239043728),


Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [58]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda x:x[1], reverse=True)

[(33, 14),
 (13, 14),
 (9, 10),
 (26, 9),
 (2, 8),
 (11, 8),
 (10, 8),
 (41, 8),
 (22, 8),
 (34, 8),
 (39, 8),
 (5, 7),
 (35, 7),
 (30, 7),
 (19, 7),
 (14, 7),
 (50, 7),
 (21, 6),
 (47, 6),
 (18, 6),
 (37, 6),
 (49, 6),
 (25, 6),
 (44, 6),
 (20, 5),
 (48, 5),
 (16, 5),
 (4, 5),
 (40, 5),
 (38, 5),
 (29, 5),
 (12, 5),
 (42, 5),
 (46, 5),
 (6, 4),
 (1, 4),
 (45, 4),
 (23, 4),
 (8, 4),
 (3, 4),
 (31, 3),
 (7, 3),
 (43, 3),
 (17, 3),
 (28, 3),
 (27, 3),
 (15, 2),
 (36, 2),
 (32, 2),
 (24, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
